@# Fine-Resolution Graph Creation from PostGIS

This notebook demonstrates advanced maritime navigation graph creation using high-resolution grids and H3 hexagonal tessellation.

The workflow covers:
1. **Base Route Loading**: Import an existing base route to focus graph creation on relevant areas.
2. **Route Buffering**: Create a buffer zone around the route to define the area of interest.
3. **Area Slicing**: Optionally reduce the AOI to a specific geographic region for testing/optimization.
4. **Fine Grid Generation**: Create high-density navigable grids (0.1-0.3 NM spacing) or H3 hexagonal grids.
5. **Graph Construction**: Build NetworkX graphs with connectivity analysis and component filtering.
6. **Efficient Persistence**: Save graphs to GeoPackage, PostGIS (standard), or PostGIS (optimized bulk insert).
7. **Route Calculation**: Compute optimal paths on the fine-resolution graph.

This notebook builds upon the base graph workflow by adding:
- **Higher resolution grids** for precise routing near coastlines, channels, and harbors
- **H3 hexagonal tessellation** as an alternative to regular square grids
- **Performance optimization** techniques for handling large graphs (millions of nodes/edges)
- **Multiple save strategies** to balance speed vs database integration

---

## Required Data

This notebook requires:
1. **Base Route**: Pre-computed route from base graph workflow (stored in PostGIS `routes` schema)
2. **ENC Data**: S-57 charts converted to PostGIS format (same as base graph requirements)
3. **Graph Configuration**: YAML config file defining layers and H3 settings
4. **Schema/Tables**:
   - Route schema: `routes` (base route table)
   - Graph schema: `graph` (for saving graph tables)
   - ENC schema: `us_enc_all` (or your schema name)
5. **PostgreSQL Environment**: Connection parameters configured in `.env` file

**Setup Instructions:**
See `docs/SETUP.md` for converting S-57 charts to PostGIS backend.

**Troubleshooting:**
If you encounter issues, see `docs/TROUBLESHOOTING.md` for common problems and solutions.

In [ ]:
# =================================================================
# Notebook Settings
# =================================================================
# This cell contains all configurable parameters for the notebook.
# Modify these settings to customize the graph creation workflow.

# --- General Workflow Flags ---
# Toggle major workflow steps on/off for testing or partial runs
buffer_sliced = True           # If True, slice buffer to reduce area (for testing/optimization)
graph_mode = "fine"            # Graph type: "fine" (regular grid) or "h3" (hexagonal tessellation)
save_gpkg = False              # Save graph to GeoPackage file (fastest, most portable)
save_pg = False                # Save graph to PostGIS using standard method
save_pg_opt = True            # Save graph to PostGIS using optimized bulk insert
calc_route = True             # Calculate and visualize a route on the created graph

# --- Data Sources & Paths ---
# PostGIS schema and table names for input data
db_schema_us_enc = "enc_west"        # Schema containing S-57 ENC layers
base_route_name = "base_route"         # Name of the base route to load
base_route_schema = "routes"           # Schema containing route tables
base_route_table = "base_routes"       # Table containing the base route
config_file_path = "src/nautical_graph_toolkit/data/graph_config.yml"  # YAML config for layers/H3 settings

# --- Geographic & Route Parameters ---
route_buffer_size_nm = 24.0           # Buffer distance around route in nautical miles
slice_south_degree = 37.0             # Southern latitude limit for buffer slicing (if buffer_sliced=True)
departure_port_name = "Pilot"         # Starting point name (from port database or custom)
arrival_port_name = "San Francisco"   # Ending point name

# --- Fine Graph Settings ---
# Parameters for regular grid graph creation
fine_grid_spacing_nm = 0.2            # Node spacing in nautical miles (smaller = denser graph)
fine_grid_max_points = 1000000        # Safety limit to prevent excessive memory usage
fine_graph_max_edge_factor = 3.0      # Max edge length multiplier (2.0 = edges up to 2x spacing)
fine_graph_bridge_components = True  # Bridge disconnected components (useful for spacing <0.1 NM)

# --- H3 Graph Settings ---
# Parameters for hexagonal tessellation graph (if graph_mode="h3")
route_schema_name = "routes"          # Schema for saving route data
graph_schema_name = "graph"           # Schema for saving graph tables (PostGIS only)
keep_largest_component = True         # Keep only the largest connected component (prevents isolated nodes)

def _to_str(value_to_format: float) -> str:
    """
    Helper function to format a numeric value for file naming.

    It converts a float into a string representation of the value multiplied
    by 100, formatted to at least two digits with leading zeros. This is
    useful for creating consistent file suffixes from grid spacing values.

    Examples:
    - 0.1   -> "10"
    - 0.05  -> "05"
    - 1.0   -> "100"
    - 0.25  -> "25"

    Args:
        value_to_format (float): The numeric value to format.

    Returns:
        str: A string representation of the value * 100, zero-padded to at least 2 digits.
    """
    # 1. Multiply by 100 to shift the first two decimal places.
    #    Example: 0.05 -> 5.0; 0.1 -> 10.0; 1.0 -> 100.0
    value = value_to_format * 100

    # 2. Round to handle potential floating-point inaccuracies and convert to integer.
    #    Example: 5.0 -> 5; 10.0 -> 10; 100.0 -> 100
    int_value = int(round(value))

    # 3. Format the integer to a two-digit string with a leading zero if needed.
    #    Example: 5 -> "05"; 10 -> "10"; 100 -> "100"
    return f"{int_value:02d}"

# --- Output & Saving ---
# Naming suffixes for output files and database tables
gpkg_h3_name_suffix = "pg_6_11"               # Suffix for H3 GeoPackage files
pg_h3_name_suffix = "pg_6_11"                 # Suffix for H3 PostGIS tables
pg_opt_h3_name_suffix = "opt_pg_6_11"         # Suffix for optimized H3 PostGIS tables
fine_grid_name_suffix = _to_str(fine_grid_spacing_nm)  # Auto-generated from spacing (e.g., "20" for 0.2nm)
pg_drop_existing = True                       # Drop existing tables before saving (be careful!)
pg_opt_chunk_size = 500000                    # Batch size for optimized PostGIS inserts


## 2. Environment Setup and Imports

This section sets up the Python environment and imports all necessary libraries for graph creation, geospatial processing, and visualization.

In [ ]:
import os
import sys
from pathlib import Path
import time
import pandas as pd
import plotly.express as px

import plotly.graph_objects as go
import plotly.io as pio
from dotenv import load_dotenv

# --- Setup Python Environment ---
# Add the src directory to the Python path to enable module imports
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Import maritime module components
from src.nautical_graph_toolkit.core.graph import (FineGraph, GraphConfigManager,
                                            H3Graph)
from src.nautical_graph_toolkit.core.pathfinding_lite import Route
from src.nautical_graph_toolkit.core.s57_data import (ENCDataFactory,
                                                S57AdvancedConfig)
from src.nautical_graph_toolkit.utils.geometry_utils import Buffer, Slicer
from src.nautical_graph_toolkit.utils.plot_utils import PlotlyChart
from src.nautical_graph_toolkit.utils.port_utils import Boundaries, PortData

# --- Load Environment Variables ---
# Load environment variables from .env file at the project root
# This loads database credentials and API tokens
load_dotenv(project_root / ".env")
pio.renderers.default = "notebook_connected"

# --- Define Output Directory ---
# Create output directory for saving graphs and results
output_dir = Path.cwd() / 'output'
output_dir.mkdir(exist_ok=True)

# --- Define Data Source (PostGIS Database Connection) ---
# PostGIS uses dict of connection params loaded from environment
# These credentials are loaded from the .env file
db_params = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}

print(f"Output directory: {output_dir}")
print(f"PostGIS DB_name: {db_params['dbname']}")

# --- Performance Tracking ---
# Dictionary to store timing metrics for each pipeline step
performance_metrics = {}

## 3. Initialize PostGIS Data Factory

The ENCDataFactory provides a unified interface for accessing S-57 ENC data from PostGIS. It handles database connections, layer queries, and spatial filtering.

In [ ]:
# --- Initialize ENC Data Factory for PostGIS Backend ---
# The factory provides a unified interface for accessing ENC data
# from PostGIS database. It will be used throughout the notebook
# for querying navigational layers and saving results.
pg_factory = ENCDataFactory(source=db_params, schema=db_schema_us_enc)

## 4. Initialize Plotly Visualization

Create the base Plotly map that will be reused throughout the notebook for visualizing routes, buffers, grids, and other geographic data.

In [ ]:
# --- Create Base Plotly Map ---
# Initialize the plotting utility and create an interactive base map
# The Mapbox token is loaded from .env for security (not hardcoded)
# The base map will be copied for different visualizations throughout the notebook
ply = PlotlyChart()
ply_fig = ply.create_base_map(mapbox_token=os.getenv('MAPBOX_TOKEN'))
ply.plotly_base_config(ply_fig)

## 5. Base Route Loading and Area of Interest Definition

Fine-resolution graphs are typically focused on specific routes to avoid creating unnecessarily large graphs. We start by loading an existing base route and creating a buffer around it to define our area of interest.

In [ ]:
# --- Load Pre-computed Base Route from PostGIS ---
# This route was created in the base graph workflow and stored in PostGIS
# We'll use it to focus our fine-resolution graph creation on relevant areas
start_time = time.perf_counter()
base_route = pg_factory.load_route(route_name=base_route_name,
                      schema_name=base_route_schema,
                      table_name=base_route_table)

end_time = time.perf_counter()
performance_metrics['Load Base Route'] = end_time - start_time
print(f"Loading base route took: {end_time - start_time:.2f}s")

# --- Visualize Base Route ---
# Create a deep copy of the base map to avoid modifying the original
# This allows us to create multiple independent map visualizations
ply_route = go.Figure(ply_fig)

ply.add_route_trace(figure=ply_route,
                    line=base_route,
                    name="Base Route")
ply_route.show()

### 5.1 Create Buffer Around Route

Create a buffer zone around the base route to define the area where we'll build our fine-resolution graph. The buffer size is configurable and typically set to 12-24 nautical miles.

In [ ]:
# --- Create Buffer Zone Around Base Route ---
# Buffer creates a polygon extending route_buffer_size_nm nautical miles
# on both sides of the route. This defines our area of interest for
# fine-resolution graph creation.
start_time = time.perf_counter()
route_buffer = Buffer.create_buffer(base_route, route_buffer_size_nm)
end_time = time.perf_counter()
performance_metrics['Create Buffer'] = end_time - start_time
print(f"Creating buffer took: {end_time - start_time:.2f}s")

In [ ]:
# --- Visualize Buffer on Map ---
# Add the buffer polygon to the map to verify it covers the desired area
ply.add_polygon_trace(fig=ply_route,
                      polygon=route_buffer,
                      name="Base Route Buffer")
ply_route.show()

### 5.2 Slice Buffer (Optional Area Reduction)

For testing or optimization, we can slice the buffer to reduce the area of interest. This is useful when experimenting with different grid spacings or graph settings without processing the entire route buffer.

In [ ]:
# --- Slice Buffer to Reduce Area (Optional) ---
# Use geographic bounding box to clip the buffer polygon
# This reduces processing time and graph size for testing
# Set buffer_sliced=False in settings to use full buffer
start_time = time.perf_counter()
sliced_buffer = Slicer.slice_by_bbox(route_buffer, south=slice_south_degree)
end_time = time.perf_counter()
performance_metrics['Slice Buffer'] = end_time - start_time
print(f"Slicing buffer took: {end_time - start_time:.2f}s")

# --- Visualize Sliced Buffer ---
# Add the sliced buffer to the map to verify the reduced area
ply.add_polygon_trace(fig=ply_route,
                      polygon=sliced_buffer,
                      name="Sliced Buffer")
ply_route.show()

## 6. Graph Creation Prerequisites

Before creating the graph, we need to:
1. Define departure and arrival points (ports or custom coordinates)
2. Load graph configuration (navigable/obstacle layers, H3 settings)
3. Filter ENCs to only those intersecting our area of interest

### 6.1 Define Departure and Arrival Points

These points will be used for routing after the graph is created. They can come from the World Port Index or be custom user-defined coordinates.

#### Port Selection: World Port Index or Custom Points

The PortData utility allows you to:
- Query ports from the World Port Index by name
- Create custom port entries with specific coordinates
- Update existing custom ports with new coordinates

In [ ]:
# --- Define Geographic Points for Routing ---
# Initialize port data utility which merges World Port Index with custom ports.
# This allows using standard ports (e.g., "San Francisco") or defining custom
# test points (e.g., "Pilot") for route calculations.
port = PortData()

# Create or update a custom port entry.
# The 'if_exists' parameter controls behavior when a port already exists:
#   - 'update': Replace coordinates with new values
#   - 'skip': Keep existing port unchanged
#   - 'raise': Throw an error if port exists
port.create_custom_port(port_name="Pilot",
                        lon=-122.27,
                        lat=37.0,
                        if_exists='update')

# Get departure and arrival points by name.
# These will be used as endpoints for route calculation.
dep_point = port.get_port_by_name(departure_port_name)
arr_point = port.get_port_by_name(arrival_port_name)

# Create detailed DataFrames for inspection (optional)
port1_df = port.get_port_details_df(dep_point)
port2_df = port.get_port_details_df(arr_point)

# Print formatted port information
print(port.format_port_string(dep_point))
print(port.format_port_string(arr_point))

# --- Visualize Ports on Map ---
# Add departure port (blue) and arrival port (red) to the map
ply.add_single_port_trace(ply_route, dep_point, name=dep_point['PORT_NAME'], color='blue')
ply.add_single_port_trace(ply_route, arr_point, name=arr_point['PORT_NAME'], color='red')
ply_route.show()

### 6.2 Load Graph Configuration and Filter ENCs

The graph configuration YAML file defines which S-57 layers to use for navigable areas and obstacles. We also filter ENCs to only those intersecting our area of interest to optimize performance.

In [ ]:
# --- Select Active Buffer (Full or Sliced) ---
# Choose which buffer polygon to use based on the buffer_sliced setting.
# Slicing is useful for:
#   - Testing workflows on smaller areas (faster iteration)
#   - Reducing memory requirements for experimentation
#   - Focusing on specific geographic regions of interest
if buffer_sliced:
    active_buffer = sliced_buffer
else:
    active_buffer = route_buffer

# --- Load Graph Configuration from YAML ---
# The config file (graph_config.yml) defines:
# - Navigable layers: Define safe water areas for routing
#   * seaare: Main sea areas from ENC charts
#   * fairwy: Designated fairways and channels
#   * drgare: Dredged areas (maintained depth)
#   * tsslpt: Traffic separation scheme lanes
#   * prcare: Precautionary areas
# - Obstacle layers: Define hazards to subtract from navigable areas
#   * lndare: Land areas (hard obstacles)
#   * slcons: Shoreline constructions (piers, breakwaters)
#   * obstrn: Obstructions (underwater hazards)
# - H3 hexagon settings: Resolution ranges and connectivity rules
#   * Multi-resolution hierarchy (typically 6-11 for coastal navigation)
#   * Bridge connectivity parameters for seamless multi-scale routing
config_path = project_root / config_file_path
config_manager = GraphConfigManager(config_path)

# --- Filter ENCs by Boundary ---
# Query PostGIS to find all ENCs that intersect our area of interest.
# This spatial filtering is CRITICAL for performance:
#   - Reduces data processing to relevant charts only
#   - Prevents loading unnecessary ENC data (thousands of charts)
#   - Typical workflow: 6000+ total ENCs → 20-50 relevant ENCs
# The method queries the enc_summary table which contains bounding boxes
# for all available charts, using PostGIS ST_Intersects for spatial filtering.
enc_list = pg_factory.get_encs_by_boundary(active_buffer)

## 7. Fine Graph Creation (Regular Grid)

Fine graphs use regular rectangular grids with dense node spacing (0.1-0.3 NM) to provide high-resolution routing. This approach is ideal for:
- Detailed coastal navigation
- Harbor and channel routing
- Areas requiring precise path planning

The process involves:
1. Creating a navigable grid from S-57 layers (similar to base graph but with finer resolution)
2. Generating a dense node network within the navigable area
3. Connecting adjacent nodes with edges
4. Filtering to keep only the largest connected component

### 7.1 Fine Grid Creation

Create a navigable grid by querying S-57 layers and combining them into a single polygon. The FineGraph class uses an iterative approach, processing usage bands in priority order to build up coverage progressively.

In [ ]:
# --- Extract Layer Configuration from YAML ---
# Get the layer settings from the config file
layers_config = config_manager.get_value("layers")

# Extract the specific lists for navigable and obstacle layers.
# These layer definitions follow IHO S-57 standard object classes:
#
# Navigable layers define safe water areas (UNION operation):
#   - seaare: General sea areas (primary coverage)
#   - fairwy: Designated fairways with maintained depth
#   - drgare: Dredged areas with known depth
#   - tsslpt: Traffic separation lanes (routing corridors)
#   - prcare: Precautionary areas (navigable but requiring caution)
#
# Obstacle layers define hazards (SUBTRACTION operation):
#   - lndare: Land areas (absolute obstacles)
#   - slcons: Shoreline constructions (piers, jetties, breakwaters)
#   - obstrn: Underwater obstructions (wrecks, rocks, etc.)
#
# The final navigable grid = (UNION of navigable) - (UNION of obstacles)
navigable_layers_config = layers_config.get('navigable', [])
obstacle_layers_config = layers_config.get('obstacles', [])

# --- Create Fine Grid ---
# Skip this section if graph_mode is set to "h3"
start_time = time.perf_counter()
if graph_mode == "fine":
    # Initialize FineGraph class with PostGIS data factory
    fg = FineGraph(data_factory=pg_factory,
                   route_schema_name=route_schema_name,
                   graph_schema_name=graph_schema_name)

    # Create the navigable grid by processing S-57 layers iteratively.
    # The method uses a prioritized approach based on ENC usage bands:
    #   Band 1: Overview (1:1,500,000+) - Coarse ocean coverage
    #   Band 2: General (1:350,000-1:1,499,999) - Coastal approaches
    #   Band 3: Coastal (1:90,000-1:349,999) - Near-shore navigation
    #   Band 4: Approach (1:30,000-1:89,999) - Harbor approaches
    #   Band 5: Harbour (1:12,000-1:29,999) - Port areas
    #   Band 6: Berthing (<1:11,999) - Detailed harbor/dock areas
    #
    # Processing order: 1→6 (coarse to fine) to progressively refine coverage.
    # Each band's features are merged with existing grid, with higher priority
    # bands overwriting lower priority areas where they overlap.
    fg_grid = fg.create_fine_grid(route_buffer=active_buffer,
                                  enc_names=enc_list,
                                  navigable_layers=navigable_layers_config,
                                  obstacle_layers=obstacle_layers_config
                                  )

    end_time = time.perf_counter()
    performance_metrics['Fine Grid Creation'] = end_time - start_time
    print(f"Fine grid creation took: {end_time - start_time:.2f}s")


### 7.2 Visualize Fine Grid Components

Plot the different components of the generated grid to verify coverage. The combined_grid (red) represents the final navigable area after merging all layers.

In [ ]:
# --- Visualize Grid Components on Map ---
# Display the grid components to verify coverage:
# - main_grid (blue): Primary sea area from 'seaare' layer
# - combined_grid (red): Final merged navigable polygon used for graph creation
if graph_mode == "fine":
    ply_fine_grid = go.Figure(ply_fig)
    ply.add_grid_trace(ply_fine_grid, name = "Main Grid", grid_geojson=fg_grid["main_grid"], color="blue")
    ply.add_grid_trace(ply_fine_grid, name = "Combined Grid", grid_geojson=fg_grid["combined_grid"], color="red")
    ply_fine_grid.show()

### 7.3 Fine Graph Construction

Convert the navigable grid polygon into a NetworkX graph by populating it with a dense network of nodes and edges. The spacing parameter controls node density (smaller = more nodes = longer processing time).

**Component Bridging for Fine Grids (<0.1 NM):**

When using very fine spacing (<0.1 NM), you may encounter artificial gaps between components due to:
- Numerical precision limits
- Slight polygon boundary misalignments
- Grid generation artifacts

The `bridge_components` parameter addresses this by:
1. Identifying disconnected components in the graph
2. Finding boundary nodes (nodes with fewer than 8 neighbors)
3. Adding bridge edges between nearby components within `max_edge_factor * spacing` distance
4. Prioritizing closest connections to maintain graph quality

**Usage:** Set `fine_graph_bridge_components=True` in settings when using spacing <0.1 NM.

In [ ]:
# --- Create Fine-Resolution Graph from Grid ---
# This is the most computationally intensive step. It creates a dense node
# network within the navigable polygon and connects adjacent nodes.
#
# Performance benchmarks (Buffer=24NM):
# - 0.3 NM spacing: ~1m 12s (158,803 nodes, 627,475 edges)
# - 0.2 NM spacing: ~2m 43s
# - 0.1 NM spacing: ~5m 59s
#
# Parameters:
# - spacing_nm: Distance between adjacent nodes in nautical miles
# - max_points: Safety limit to prevent excessive memory usage
# - max_edge_factor: Max edge length = spacing * max_edge_factor (also used for bridging)
# - bridge_components: If True, bridges disconnected components (recommended for spacing <0.1 NM)
# - keep_largest_component: Remove isolated node clusters for routing reliability
start_time = time.perf_counter()
if graph_mode == "fine":
    G_fine = fg.create_base_graph(grid_data=fg_grid["combined_grid"],
                         spacing_nm=fine_grid_spacing_nm,
                         max_points=fine_grid_max_points,
                         max_edge_factor=fine_graph_max_edge_factor,
                         bridge_components=fine_graph_bridge_components,
                         keep_largest_component=keep_largest_component)

    end_time = time.perf_counter()
    performance_metrics['Fine Graph Creation'] = end_time - start_time
    print(f"Fine graph creation took: {end_time - start_time:.2f}s")


## 8. H3 Graph Creation (Hexagonal Tessellation)

H3 graphs use Uber's H3 hexagonal tessellation system instead of regular square grids. Benefits include:
- **Uniform neighbor distances**: All adjacent hexagons are equidistant
- **Better angular coverage**: 6 neighbors vs 8 in square grids (no diagonal artifacts)
- **Hierarchical resolution**: H3's multi-resolution structure allows mixed detail levels
- **Efficient connectivity**: Natural bridging between resolution levels

The H3Graph class handles multi-resolution hexagon creation, obstacle subtraction, and connectivity between resolution levels.

### 8.1 H3 Grid and Graph Creation

Create an H3-based graph in a single operation. The method handles grid creation, obstacle subtraction, graph construction, and connectivity bridging automatically.

In [ ]:
# --- Create H3 Hexagonal Graph ---
# This section only runs if graph_mode is set to "h3"
#
# H3 Graph Advantages:
# -------------------------
# 1. Uniform neighbor distances: All adjacent hexagons are equidistant
#    (eliminates diagonal distance artifacts from square grids)
# 2. Better angular coverage: 6 neighbors vs 8 in square grids
# 3. Hierarchical resolution: Natural multi-scale structure
#    - Parent hexagons subdivide into 7 child hexagons
#    - Seamless bridging between resolution levels
# 4. Efficient connectivity: Natural connectivity across resolution scales
#
# Performance benchmarks (Buffer=24NM, Resolutions 6-11):
# - Base H3 graph: ~1m 58s (933,188 hexagons, 2,765,899 edges)
# - With bridging: ~13m 20s (3,838,809 nodes, 11,528,770 edges, 462 components)
#
# The H3 graph creation process:
# 1. Generates hexagons at multiple resolutions within navigable areas based on config
#    - Each resolution level provides different detail (coarse to fine)
#    - Resolution 6: ~36 km² per hexagon (ocean navigation)
#    - Resolution 11: ~4 m² per hexagon (harbor precision)
# 2. Subtracts obstacles from hexagon coverage (land, constructions, obstructions)
# 3. Creates graph edges between adjacent hexagons (same resolution)
# 4. Bridges between resolution levels for seamless navigation
#    - Connects coarse and fine areas without artificial boundaries
#    - Enables smooth transitions from ocean to harbor routing
# 5. Optionally keeps only the largest connected component
#    - Removes isolated nodes/islands for routing reliability

start_time = time.perf_counter()
if graph_mode == "h3":
    # Initialize H3Graph class with PostGIS data factory
    h3 = H3Graph(data_factory=pg_factory,
                route_schema_name=route_schema_name,
                graph_schema_name=graph_schema_name)

    # Load H3 settings from configuration YAML.
    # Includes resolution ranges, bridge settings, and connectivity rules:
    #   - resolution_ranges: Dict mapping usage bands to H3 resolution levels
    #   - connectivity.bridge_between_resolutions: Enable multi-scale bridging
    #   - connectivity.min_same_res_neighbors: Minimum same-res neighbors before bridging
    #   - connectivity.target_total_neighbors: Target total neighbor count with bridges
    #   - connectivity.max_bridge_distance_nm: Max distance for bridge connections
    h3_settings = config_manager.get_value("h3_settings")
    connectivity_config = h3_settings.get('connectivity', {})

    # Create H3 graph and grid in one operation.
    # Returns both the NetworkX graph and the hexagon GeoDataFrame for visualization.
    # The graph includes:
    #   - Node attributes: lon, lat, h3_index, resolution, geometry
    #   - Edge attributes: length (nautical miles), bridge (boolean flag)
    G_h3, h3_grid = h3.create_h3_graph(route_buffer=active_buffer,
                                       enc_names=enc_list,
                                       navigable_layers=navigable_layers_config,
                                       obstacle_layers=obstacle_layers_config,
                                       connectivity_config=connectivity_config,
                                       keep_largest_component=keep_largest_component)

    end_time = time.perf_counter()
    performance_metrics['H3 Graph Creation'] = end_time - start_time
    print(f"H3 graph creation took: {end_time - start_time:.2f}s")


## 9. Save Graph to Persistent Storage

The notebook provides three save strategies with different performance characteristics:
1. **GeoPackage**: Fastest, most portable, ideal for QGIS visualization
2. **PostGIS (Standard)**: Full-featured database integration, slower for large graphs
3. **PostGIS (Optimized)**: Bulk insert strategy, ~2x faster than standard method

Choose save methods based on your workflow needs.

### 9.1 Save to GeoPackage (Fastest, Most Portable)

GeoPackage format is ideal for:
- Offline analysis and visualization (QGIS, ArcGIS)
- Sharing graphs with collaborators
- Archiving graph snapshots
- Fast save/load operations (no database overhead)

In [ ]:
# --- Save Graph to GeoPackage File ---
# GeoPackage (.gpkg) is a SQLite-based format optimized for geospatial data
# It creates separate layers for nodes and edges with spatial indexes
#
# Performance benchmarks:
# - Fine graph (46K nodes, 181K edges): ~4s
# - H3 graph (3.8M nodes, 11.5M edges): ~8-10m
#
# The file can be opened in QGIS, ArcGIS, or loaded back into Python
start_time = time.perf_counter()
if save_gpkg:
    if graph_mode == "h3":
        name = f"{graph_mode}_graph_{gpkg_h3_name_suffix}.gpkg"
        h3.save_graph_to_gpkg(G_h3, output_path=output_dir/ name)
    else:
        name = f"{graph_mode}_graph_{fine_grid_name_suffix}.gpkg"
        fg.save_graph_to_gpkg(G_fine, output_path=output_dir / name)
    end_time = time.perf_counter()
    performance_metrics['Save to GeoPackage'] = end_time - start_time
    print(f"Saving to GeoPackage took: {end_time - start_time:.2f}s")

### 9.2 Save to PostGIS (Standard Method)

Standard PostGIS save provides full database integration with spatial indexes and constraints. Best for:
- Server-side spatial queries
- Integration with other PostGIS workflows
- Smaller to medium graphs (<500K nodes)

In [ ]:
# --- Save Graph to PostGIS Database (Standard Method) ---
# Uses SQLAlchemy to insert nodes and edges with full database integration
# Creates spatial indexes automatically for fast spatial queries
#
# Performance benchmarks:
# - Fine graph (46K nodes, 181K edges): ~7-8s
# - Medium graph (357K nodes, 1.4M edges): ~4m
# - Large H3 graph (3.8M nodes, 11.5M edges): ~30-40m
#
# WARNING: Setting pg_drop_existing=True will delete existing tables!
start_time = time.perf_counter()
if save_pg:
    if graph_mode == "h3":
        name = f"{graph_mode}_graph_{pg_h3_name_suffix}"
        h3.save_graph_to_postgis(graph=G_h3, table_prefix= name, drop_existing=pg_drop_existing)
    else:
        name = f"{graph_mode}_graph_{fine_grid_name_suffix}"
        fg.save_graph_to_postgis(graph=G_fine, table_prefix= name, drop_existing=pg_drop_existing)
    end_time = time.perf_counter()
    performance_metrics['Save to PostGIS (Original)'] = end_time - start_time
    print(f"Saving to PostGIS (Original) took: {end_time - start_time:.2f}s")

### 9.3 Save to PostGIS (Optimized Bulk Insert)

Optimized method uses PostgreSQL COPY for bulk inserts, achieving ~2x speedup over standard method. Best for:
- Large graphs (>500K nodes)
- Batch processing multiple graphs
- Performance-critical workflows

In [ ]:
# --- Save Graph to PostGIS Database (Optimized Method) ---
# Uses PostgreSQL COPY command for bulk inserts, significantly faster for large graphs
# Processes data in chunks to manage memory efficiently
#
# Performance benchmarks:
# - Fine graph (46K nodes, 181K edges): ~7-8s
# - Medium graph (357K nodes, 1.4M edges): ~2m (vs 4m standard)
# - Large graph (1.4M nodes, 5.7M edges): ~2m (vs 4-5m standard)
#
# chunk_size controls batch size (500K is optimal for most cases)
start_time = time.perf_counter()
if save_pg_opt:
    if graph_mode == "h3":
        name = f"{graph_mode}_graph_{pg_opt_h3_name_suffix}"
        h3.save_graph_to_postgis_optimized(graph=G_h3, table_prefix= name, drop_existing=pg_drop_existing, chunk_size=pg_opt_chunk_size)
    else:
        name = f"{graph_mode}_graph_opt_{fine_grid_name_suffix}"
        fg.save_graph_to_postgis_optimized(graph=G_fine, table_prefix= name, drop_existing=pg_drop_existing, chunk_size=pg_opt_chunk_size)
    end_time = time.perf_counter()
    performance_metrics['Save to PostGIS (Optimized)'] = end_time - start_time
    print(f"Saving to PostGIS (Optimized) took: {end_time - start_time:.2f}s")

## 10. Route Calculation on Fine Graph

With the fine-resolution graph created, we can now compute optimal routes with higher precision than the base graph. The finer grid spacing provides:
- More accurate coastline following
- Better channel navigation
- Improved route optimization in complex areas

### 10.1 Compute Route Using A* Algorithm

The Route class uses the A* pathfinding algorithm to find the shortest path between departure and arrival points. The algorithm considers only distance in this base route calculation (weights can be added later).

In [ ]:
# --- Calculate Route on Fine-Resolution Graph ---
# Use the A* algorithm to find the shortest path between departure and arrival points.
#
# A* Algorithm Benefits:
# ----------------------
# 1. Optimal pathfinding: Guaranteed to find shortest path (with admissible heuristic)
# 2. Efficient search: Uses heuristic to guide exploration toward goal
# 3. Graph-based: Works on any connected graph structure (fine grid or H3)
#
# The routing process:
# 1. Maps user-provided port coordinates to nearest graph nodes
# 2. Applies A* with Euclidean distance heuristic
# 3. Returns route geometry (LineString) and total distance (nautical miles)
#
# Current implementation uses base distance weighting (all edges weighted equally
# by length). Future enhancements can add:
#   - Traffic separation scheme priorities
#   - Depth-based routing (avoid shallow areas)
#   - Weather/current integration
#   - Vessel-specific constraints (draft, size)

start_time = time.perf_counter()
if calc_route:
    # Get the departure and arrival port geometries
    dep_point = port.get_port_by_name(departure_port_name)
    arr_point = port.get_port_by_name(arrival_port_name)

    # Select the appropriate graph based on mode
    if graph_mode == 'h3':
        graph_for_routing = G_h3
    else:
        graph_for_routing = G_fine

    # Initialize Route class with the graph and data manager.
    # The data manager provides database connectivity for saving/loading routes.
    route = Route(graph=graph_for_routing, data_manager=pg_factory.manager)
    
    # Compute the route using A* algorithm.
    # The method automatically:
    #   1. Maps port coordinates to nearest graph nodes using spatial index
    #   2. Validates start/end nodes are in the graph
    #   3. Runs A* pathfinding with distance-based weighting
    #   4. Converts node sequence to route geometry (LineString)
    #   5. Calculates total distance in nautical miles
    route_geometry, distance = route.base_route(
        departure_point=dep_point.geometry,
        arrival_point=arr_point.geometry,
    )

    end_time = time.perf_counter()
    performance_metrics['Route Calculation'] = end_time - start_time
    print(f"Route calculation took: {end_time - start_time:.2f}s")


### 10.2 Visualize Computed Route

Display the calculated route on an interactive map to verify the path makes sense and follows navigable channels appropriately.

In [ ]:
# --- Visualize Route on Interactive Map ---
# Create a new map figure and add the route, departure port, and arrival port
if calc_route:
    ply_fine_route = go.Figure(ply_fig)
    # Add the computed route as a line
    ply.add_route_trace(figure=ply_fine_route,
                        line=route_geometry,
                        name="Base Route")
    # Add departure port marker (blue)
    ply.add_single_port_trace(ply_fine_route, dep_point, name=dep_point['PORT_NAME'], color='blue')
    # Add arrival port marker (red)
    ply.add_single_port_trace(ply_fine_route, arr_point, name=arr_point['PORT_NAME'], color='red')
    ply_fine_route.show()


## 11. Performance Summary and Benchmark Export

This section visualizes the time taken for each step of the pipeline and exports detailed performance metrics to CSV for long-term tracking and analysis.

In [ ]:
# --- Visualize Pipeline Performance Metrics ---
# Create an interactive bar chart showing time taken for each pipeline step.
# This helps identify bottlenecks and optimize the workflow for different scenarios.
if performance_metrics:
    # Convert the performance metrics dictionary to a pandas DataFrame
    perf_df = pd.DataFrame(list(performance_metrics.items()), columns=['Step', 'Time (seconds)'])
    # Sort by time taken (longest first) to highlight bottlenecks
    perf_df = perf_df.sort_values(by='Time (seconds)', ascending=False)

    # Create an interactive bar chart with time values displayed on bars
    fig = px.bar(
        perf_df,
        x='Step',
        y='Time (seconds)',
        title='Fine Graph Creation Pipeline Performance (PostGIS)',
        text_auto='.2f',  # Show values with 2 decimal places
        labels={'Step': 'Pipeline Step', 'Time (seconds)': 'Time Taken (seconds)'}
    )
    # Position labels outside bars for better readability
    fig.update_traces(textposition='outside')
    fig.show()
else:
    print("No performance metrics were recorded. Run the notebook cells to generate the summary.")

# --- Export Performance Benchmarks to CSV ---
# Save detailed performance metrics to CSV for long-term tracking and analysis
# This allows comparison across different runs, configurations, and hardware
if performance_metrics:
    from datetime import datetime
    
    # Determine graph type and get node/edge counts
    if graph_mode == "h3":
        final_graph = G_h3
    else:
        final_graph = G_fine
    
    node_count = final_graph.number_of_nodes()
    edge_count = final_graph.number_of_edges()
    
    # Calculate normalized metrics (per 100K nodes)
    time_per_100k_nodes = {}
    if node_count > 0:
        for step, time_val in performance_metrics.items():
            time_per_100k_nodes[step] = (time_val / node_count) * 100000
    
    # Build benchmark record with metadata and metrics
    benchmark_record = {
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'workflow': 'graph_fine_PostGIS_v2',
        'graph_mode': graph_mode,
        'data_source': 'PostGIS',
        'db_schema': db_schema_us_enc,
        'node_count': node_count,
        'edge_count': edge_count,
        'spacing_nm': fine_grid_spacing_nm if graph_mode == "fine" else 'h3',
        'buffer_size_nm': route_buffer_size_nm,
        'buffer_sliced': buffer_sliced,
        'keep_largest_component': keep_largest_component,
        # Individual timing metrics
        'load_base_route_sec': performance_metrics.get('Load Base Route', 0),
        'create_buffer_sec': performance_metrics.get('Create Buffer', 0),
        'slice_buffer_sec': performance_metrics.get('Slice Buffer', 0),
        'fine_grid_creation_sec': performance_metrics.get('Fine Grid Creation', 0),
        'fine_graph_creation_sec': performance_metrics.get('Fine Graph Creation', 0),
        'h3_graph_creation_sec': performance_metrics.get('H3 Graph Creation', 0),
        'save_gpkg_sec': performance_metrics.get('Save to GeoPackage', 0),
        'save_postgis_original_sec': performance_metrics.get('Save to PostGIS (Original)', 0),
        'save_postgis_optimized_sec': performance_metrics.get('Save to PostGIS (Optimized)', 0),
        'route_calculation_sec': performance_metrics.get('Route Calculation', 0),
        # Normalized metrics (time per 100K nodes)
        'grid_creation_per_100k_nodes': time_per_100k_nodes.get('Fine Grid Creation', 0) or time_per_100k_nodes.get('H3 Graph Creation', 0),
        'graph_creation_per_100k_nodes': time_per_100k_nodes.get('Fine Graph Creation', 0),
        'save_gpkg_per_100k_nodes': time_per_100k_nodes.get('Save to GeoPackage', 0),
        'save_postgis_orig_per_100k_nodes': time_per_100k_nodes.get('Save to PostGIS (Original)', 0),
        'save_postgis_opt_per_100k_nodes': time_per_100k_nodes.get('Save to PostGIS (Optimized)', 0),
        # Total pipeline time
        'total_pipeline_sec': sum(performance_metrics.values()),
    }
    
    # Convert to DataFrame
    benchmark_df = pd.DataFrame([benchmark_record])
    
    # Define CSV file path (one file per workflow type)
    benchmark_csv = output_dir / 'benchmark_graph_fine.csv'
    
    # Append to existing CSV or create new one
    if benchmark_csv.exists():
        # Load existing data and append
        existing_df = pd.read_csv(benchmark_csv)
        combined_df = pd.concat([existing_df, benchmark_df], ignore_index=True)
        combined_df.to_csv(benchmark_csv, index=False)
        print(f"Appended benchmark to existing file: {benchmark_csv}")
        print(f"Total benchmark records: {len(combined_df)}")
    else:
        # Create new CSV file
        benchmark_df.to_csv(benchmark_csv, index=False)
        print(f"Created new benchmark file: {benchmark_csv}")
    
    # Display the current benchmark record
    print("\n=== Current Benchmark Record ===")
    print(f"Timestamp: {benchmark_record['timestamp']}")
    print(f"Graph Mode: {benchmark_record['graph_mode']}")
    print(f"Data Source: {benchmark_record['data_source']}")
    print(f"Nodes: {benchmark_record['node_count']:,}")
    print(f"Edges: {benchmark_record['edge_count']:,}")
    print(f"Total Pipeline Time: {benchmark_record['total_pipeline_sec']:.2f}s")
    print(f"\nMost demanding operations:")
    
    # Show top 3 time-consuming operations
    top_operations = sorted(
        [(k, v) for k, v in performance_metrics.items()],
        key=lambda x: x[1],
        reverse=True
    )[:3]
    for i, (op, time_val) in enumerate(top_operations, 1):
        print(f"  {i}. {op}: {time_val:.2f}s")
else:
    print("No performance metrics to export.")